# Layer 1 — Decision Definition

**Objective:** Define the business decision (pLTV30 for UA seed optimization),
extract KPIs from the dataset, and output `decision_definition.md`.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path
import sys

ROOT = Path('.').resolve().parent
sys.path.insert(0, str(ROOT))
from utils.reporting import write_report, md_table, md_section, timestamp_line

DATA_PATH = ROOT / 'data' / 'cfm_pltv.csv'
if not DATA_PATH.exists():
    DATA_PATH = ROOT / 'data' / 'cfm_pltv_sample.csv'
df = pd.read_csv(DATA_PATH)
print(f'Loaded {len(df):,} rows from {DATA_PATH.name}')

In [ ]:
# Key KPIs
arpu = df['ltv30'].mean()
pay_rate = df['is_payer_30'].mean() * 100
payers = df[df['rev_d7'] > 0]
d7_d30_mult = (payers['ltv30'] / payers['rev_d7'].clip(lower=0.01)).median() if len(payers) > 0 else 0
median_ltv_payers = df[df['ltv30'] > 0]['ltv30'].median()
top1_n = max(1, len(df) // 100)
top1_rev_share = df.nlargest(top1_n, 'ltv30')['ltv30'].sum() / df['ltv30'].sum() * 100 if df['ltv30'].sum() > 0 else 0

print(f'ARPU (D30):          ${arpu:.2f}')
print(f'Paying Rate (D30):   {pay_rate:.1f}%')
print(f'D7→D30 Multiplier:   {d7_d30_mult:.1f}×')
print(f'Median LTV30 (payers): ${median_ltv_payers:.2f}')
print(f'Top-1% Revenue Share:  {top1_rev_share:.0f}%')

In [ ]:
# LTV30 Distribution
fig = px.histogram(df[df['ltv30'] > 0], x='ltv30', nbins=50,
                   title='LTV30 Distribution (Payers, Log Scale)',
                   log_y=True, labels={'ltv30': 'LTV30 ($)'})
fig.show()

In [ ]:
# Generate report
report = f"""# Layer 1 — Decision Definition

{timestamp_line()}

## Business Decision

**Objective:** Predict player Lifetime Value at 30 days (pLTV30) to optimize UA seed lists.

## Key Performance Indicators

{md_table(
    ['KPI', 'Value'],
    [
        ['ARPU (D30)', f'${arpu:.2f}'],
        ['Paying Rate (D30)', f'{pay_rate:.1f}%'],
        ['D7→D30 Multiplier', f'{d7_d30_mult:.1f}×'],
        ['Median LTV30 (payers)', f'${median_ltv_payers:.2f}'],
        ['Top-1% Revenue Share', f'{top1_rev_share:.0f}%'],
    ]
)}
"""

write_report('decision_definition.md', report)
print('Done!')